In [1]:
import pandas as pd
import numpy as np
import random
import time
from collections import Counter

In [3]:
#load file
from google.colab import files
uploaded = files.upload()

Saving WineQT.csv to WineQT.csv


In [4]:
#-- Convert csv to dataframe
df=pd.read_csv("WineQT.csv")
df=df.drop("Id",1)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
#-- Function to standardize data
def standardization(series,target):
  if series.name != target:
    return (series - series.mean()) / series.std()
  else:
    return series

In [6]:
#-- Show dataframe characteristics
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1143 non-null   float64
 1   volatile acidity      1143 non-null   float64
 2   citric acid           1143 non-null   float64
 3   residual sugar        1143 non-null   float64
 4   chlorides             1143 non-null   float64
 5   free sulfur dioxide   1143 non-null   float64
 6   total sulfur dioxide  1143 non-null   float64
 7   density               1143 non-null   float64
 8   pH                    1143 non-null   float64
 9   sulphates             1143 non-null   float64
 10  alcohol               1143 non-null   float64
 11  quality               1143 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 107.3 KB


In [18]:
#-- KNN version using numpy in order to gain some speed
def knn(dfx,dfy,k,target,weights=[1,1,1,1,1,1,1,1,1,1,1]):
  res=[]
  correct=[]
  for  y in dfy:
    res=[]
    temp=[]
    for x in dfx:
      dist=sum(np.absolute(x[:-1] - y[:-1])*np.array(weights))
      #dist=sum(((x[:-1] - y[:-1])**2)*np.array(weights))
      res.append([dist,x[-1]])
    temp=sorted(res, key=lambda x: x[0],reverse=False)[:k]
    freqs = Counter([x[1] for x in temp])
    max_key = max(freqs, key=freqs.get)
    correct.append(max_key==y[-1])
  return round(sum(correct)/len(correct)*100, 2)


In [21]:
#-- One indiv is one knn, with his weights and accuracy score
class Indiv:
    def __init__(self,weights,k,dfx,dfy,target,index):
        self.k=k
        self.weights=weights
        self.target=target
        self.score=knn(dfx,dfy,self.k,target,self.weights)
        self.index=index
    def __repr__(self):
        return f"weights={self.weights} / k={self.k} / index={self.index}/ Accuracy={self.score}%"
    def __eq__(self,autre):
        if(self.weights==autre.weights and self.k==autre.k):
            return True
        else:
            return False
    def recalcule(self):
        self.score=knn(dfx,dfy,self.k,self.target,self.weights)


#-- Function to initialize weights and k parameter, randomly
def aleat(df):
  size=len(df.columns)-1
  w=[]
  for i in range(size):
    w.append(round(random.uniform(0,1),2))
  k=random.randint(3,21)
  return k,w

#-- Create multiple distincts knn
def creation_pop(nbr_indiv,df,target,dfx,dfy):
    pop=[]
    for x in range(nbr_indiv):
        k,weights=aleat(df)
        z=Indiv(weights,k,dfx,dfy,target,x+1)
        pop.append(z)
    return pop

#-- Function sorting knn based on their accuracy score
def tri(liste):
    return sorted(liste,key=lambda s: s.score,reverse=True)

#-- Function to change randomly weights or k on knn on one specific knn
def mutation_aleat(pop,index=0):
    y=random.randint(0,1)
    weights=pop[0].weights
    if(index==0):
        indiv=random.choice(pop)
        print("mutation aléatoire sur individu numéro =",indiv.index)
    else:
        indiv=next((x for x in pop if x.index == index), None)
        print("mutation aléatoire sur individu numéro =",indiv.index)
    if(y==1):
        indiv.k=random.randint(3,21)
    else:
        x=random.randint(0,len(weights)-1)
        indiv.weights[x]=round(random.uniform(0,1),2)

#-- Function to put paremeters from best knn on others, randomly, in order to create new combinations
def croisement(best,nbr_parents):
    tot=[]
    weights=best[0].weights
    for i in range(len(weights)):
      sub=[]
      for j in range(nbr_parents):
        sub.append(best[j].weights[i])
      tot.append(sub)
    k=[]
    for j in range(nbr_parents):
        k.append(best[j].k)
    for i in range(nbr_parents,len(best)):
        best[i].k=random.choice(k)
        x=random.randint(0,len(weights)-1)
        y=random.randint(0,len(tot[0])-1)
        best[i].weights[x]=tot[x][y]
    return best

#-- Function to print results    
def affich_pop(pop,index=0):
    if(index!=0 and index<len(pop)):
        for i in range(index):
            print(pop[i])
    else:
        for individu in pop:
            print(individu)

#-- Function to change parameters if two knn are the same
def mut_egaux(pop):
    for i in range(len(pop)):
        for j in range(len(pop)):
            rep=pop[i]==pop[j]
            if(rep==True and i!=j):
                x=pop[j].index
                mutation_aleat(pop,x)

#-- Function to change randomly parameters on multiple knn
def random_mut(best,size):
  weights=len(best[0].weights)
  for i in range(size,len(best)):
    print("Random mutation on indiv ", best[i].index)
    x=random.randint(0,weights-1)
    y=random.randint(0,weights-1)
    best[i].weights[x]=round(random.uniform(0,1),2)
    #best[i].weights[y]=round(random.uniform(0,1),2)
  t=random.randint(size,len(best)-1)
  print("K mutation on indiv",best[t].index)
  best[t].k=random.randint(3,21)

#-- Main function, optimizing parameters of knn
def algo_gen(nbr_indiv,taux_mut,nbr_gen,df,dfx,dfy,target):
    pop=[]
    pop=creation_pop(nbr_indiv,df,target,dfx,dfy)
    best=tri(pop)
    res=best[0].score
    n=0
    print("génération=",n)
    affich_pop(best)
    print(" ")
    while(n<nbr_gen):
        n=n+1
        parents=int(len(pop)/2)
        best=croisement(best,parents)
        print("génération =",n)
        mut=random.uniform(0,1)
        if(mut<taux_mut):
            mutation_aleat(best)
        #mut_egaux(best)
        size=int(len(pop)*3/4)
        random_mut(best,size)
        for i in best:
          i.recalcule()    
        best=tri(best)
        affich_pop(best)
        print(" ")
        res=best[0].score

In [23]:
#-- Run
start=time.clock()
target='quality' #column we want to predict
df = df.reindex(columns = [col for col in df.columns if col != target] + [target])  #it's important to put our target column at the last position 
dfx=df.sample(frac=0.75)  #training set represents 75% of the original dataset
dfy = df.loc[~df.index.isin(dfx.index)]  #test set is all rows unused by training set

#-- We standardize or dataset
for i in dfx.columns:
  dfx[i]=standardization(dfx[i],target)
  dfy[i]=standardization(dfy[i],target)

#-- We convert dataframes to numpy arrays in order to be faster
dfx=dfx.to_numpy()
dfy=dfy.to_numpy()

#-- We initialize operation with 20 distinct knn, 80% of chances to have an unique parameter change at each iteration, and we stop when we reach 100 iterations
algo_gen(20,0.8,100,df,dfx,dfy,target)
print(time.clock()-start,"seconds")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


génération= 0
weights=[0.13, 0.95, 0.42, 0.54, 0.68, 0.98, 0.13, 0.32, 0.08, 0.64, 0.46] / k=8 / index=9/ Accuracy=61.19%
weights=[0.29, 0.21, 0.19, 0.67, 0.64, 0.6, 0.66, 0.41, 0.24, 0.99, 0.38] / k=12 / index=4/ Accuracy=58.04%
weights=[0.73, 0.54, 0.95, 0.44, 0.5, 0.69, 0.43, 0.53, 0.13, 0.91, 0.51] / k=13 / index=10/ Accuracy=57.69%
weights=[0.25, 0.02, 0.3, 0.73, 0.1, 0.07, 0.83, 0.7, 0.6, 0.56, 0.26] / k=13 / index=6/ Accuracy=56.99%
weights=[0.02, 0.95, 0.96, 0.09, 0.48, 0.49, 0.11, 0.27, 0.75, 0.81, 0.53] / k=21 / index=8/ Accuracy=56.64%
weights=[0.2, 0.35, 0.69, 0.46, 0.95, 0.19, 0.65, 0.22, 0.98, 0.97, 0.79] / k=19 / index=11/ Accuracy=56.64%
weights=[0.29, 0.03, 0.9, 0.41, 0.55, 0.57, 0.03, 0.15, 0.54, 0.77, 0.29] / k=11 / index=18/ Accuracy=56.64%
weights=[0.14, 0.77, 0.28, 0.96, 0.76, 0.76, 0.61, 0.07, 0.58, 0.46, 0.84] / k=6 / index=3/ Accuracy=56.29%
weights=[0.25, 0.4, 0.39, 0.49, 0.25, 0.11, 0.03, 0.05, 0.56, 0.39, 0.35] / k=8 / index=16/ Accuracy=56.29%
weights=[0.04

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


We obtain an accuracy score of 62,59%. For information, without parameter optimization (all weights equal to 1 and k=5) we obtain a score near to 50% as we can see in the next cell

In [ ]:
#-- First version of my knn using pandas
#-- it works, but takes 20 times much than the numpy version
def knn_v1(dfx,dfy,k,target,weights=[1,1,1,1,1,1,1,1,1,1,1]):
  tempx=dfx.loc[:, dfx.columns != target]
  tempy=dfy.loc[:, dfy.columns != target]
  for i in tempx.columns:
    tempx[i]=standardization(tempx[i])
    tempy[i]=standardization(tempy[i])
  dfx[tempx.columns]=tempx*weights
  dfy[tempy.columns]=tempy*weights
  

  dfy["vote"]=np.nan
  for i in range(len(dfy)):
    dff=dfx.copy()
    df2=dfy[dfy.columns.difference(["vote",target])].copy()
    first_row = df2.iloc[[i]].values[0]
    dff[dff.columns.difference([target])]=dff[dff.columns.difference([target])].apply(lambda row: row - first_row, axis=1)
    dff[dff.columns.difference([target])]=dff[dff.columns.difference([target])].abs()
    dff["distance"]=dff[dff.columns.difference([target])].sum(axis=1)
    dff=dff.sort_values("distance").head(k)
    top=dff[target].value_counts()
    top=top.reset_index().sort_values(target,ascending=False)
    dfy["vote"].iloc[i]=top["index"].iloc[0]
    #print(dff)
  dfy["Correct"]=np.nan
  dfy["Correct"]=dfy[target]==dfy["vote"]
  score=round(dfy["Correct"].sum()/len(dfy["Correct"])*100 , 2)
  return score

dfx=df.loc[:900,:]
dfy=df.loc[900:,:]
rep=knn(dfx,dfy,5,"quality")
rep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

50.21